# Alibaba 
## Data Aquisition

In [ ]:
import pandas as pd
from deepctr.data.base import DagBuilder
from deepctr.utils.io import YamlIO

In [ ]:
mode = 'production'
config_filepath = "config/alibaba.yml"
yaml = YamlIO()
config = yaml.read(config_filepath)
builder = DagBuilder(config=config)
dag = builder.build()
dag.run()
